In [1]:
import pandas as pd
import numpy as np

dft = pd.read_csv("complete_times.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("complete_distances.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [2]:
def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)
def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

In [104]:
table=pd.read_csv("route.csv", delimiter=";")
time_table=pd.read_csv("route_time.csv", delimiter=";")

In [4]:
import datetime
def difference(day, index):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index+1], '%H:%M')
    return ((time2-time).seconds)/60

In [151]:
gained_time1=0
gained_time2=0 
days = ['martedi', 'mercoledi','giovedì']
#for day in days:
day='martedi'
for i in range(len(time_table[day].dropna())-2):
    try:
        diff=difference(day, i)
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][i+1]
        value=row.loc[:,get_column(dft, paese).values]
        gained_time1=gained_time1+(diff-float(value.values[0][0]))
        gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if diff>(float(value.values[0][0])-gained_time1):
            print(f"{day}: primo : {paese} : {gained_time1}")
            gained_time2=(gained_time2-(diff-float(value.values[0][0])))+diff
        elif diff>float(value.values[0][0])-gained_time2:
            print(f"{day}: secondo : {paese}: {gained_time2}")
            gained_time1=(gained_time1-(diff-float(value.values[0][0])))+diff
        else:
            print(f"{day}: impossible : {paese} : {gained_time1} : {gained_time2}")
            gained_time1=(gained_time1-(diff-float(value.values[0][0])))+diff
            gained_time2=(gained_time2-(diff-float(value.values[0][0])))+diff
        #print(f"{diff} ^ {float(value.values[0][0])} ^ {gained_time1} ^ {gained_time2} ^ {diff>(float(value.values[0][0])-gained_time1)}")
        print(f"{diff>(float(value.values[0][0])-gained_time1)}")
        
    except:
        print(value)
    #if diff<dft[row[paese].values]:
    #    print("Not acceptable!")

martedi: primo : Cremona, Felice Cavallotti, 2 : 56.6
True
martedi: primo : Casalpusterlengo : 37.5
True
martedi: secondo : Cremona, Ugolani Dati, 4: 85.45
True
martedi: primo : Crema, Civerchi : 27.016666666666666
True
martedi: secondo : Bonemerse: 60.06666666666667
False
martedi: impossible : Sospiro : -71.06666666666666 : -43.01666666666666
False
martedi: impossible : Vailate : -62.366666666666674 : -34.31666666666668
False
martedi: impossible : Pianengo : -243.28333333333336 : -215.23333333333335
False
martedi: impossible : Vescovato : -193.65000000000003 : -165.60000000000002
False
martedi: primo : Offanengo : 42.16666666666665
True
martedi: impossible : Piadena : -18.08333333333335 : 59.81666666666666
False
martedi: primo : Gussola : 48.98333333333332
True
martedi: primo : Solarolo : 204.89999999999998
True
martedi: primo : Ostiano : 181.26666666666665
True
martedi: primo : Scandolara Ravara, Italia : 182.95
True
martedi: primo : Persico Dosimo : 153.91666666666666
True
martedi: 

In [107]:
diff=difference("martedi", 15)
row=dft[get_row(dft, table["martedi"][0]).values]
paese=table["martedi"][15]
value=row.loc[:,get_column(dft, paese).values]
#gained_time=gained_time+(diff-float(value.values[0][0]))

In [124]:
diff=difference("lunedi", 0)

row=dft[get_row(dft, table["lunedi"][0]).values]
value=row.loc[:,get_column(dft, "bagnolo").values]
float(value.values[0][0])
diff

10.0